In [15]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
import matplotlib.pyplot as plt


def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

from enum import IntEnum

class CallPutType(IntEnum):
    '''define a class of CallPutType'''
    Call = 0
    Put = 1
class EuroAmerType(IntEnum):
    EuropeanOption = 0
    AmericanOption = 1
    
def delta_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    if call_put_type == CallPutType.Call:
        delta = np.exp(-q*T) * norm.cdf(d1_val)
    elif call_put_type == CallPutType.Put:
        delta = -np.exp(-q*T) * norm.cdf(-d1_val)
    else:
        raise ValueError("invalid call_put_type")
    return delta

def gamma_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    # gammas of C and P are the same for european options
    gamma = np.exp(-q*T) / (S*sigma*np.sqrt(T)) * (1/np.sqrt(2*np.pi)) * np.exp(-d1_val**2/2)
    return gamma

def vega_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    # vegas of C and P are the same for european options
    vega = S * np.exp(-q*T) * np.sqrt(T) * (1/np.sqrt(2*np.pi)) * np.exp(-d1_val**2/2)
    return vega 

def theta_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma)
    if call_put_type == CallPutType.Call:
        theta = (
            -(S*sigma*np.exp(-q*T)) / (2*np.sqrt(2*np.pi*T)) * np.exp(-d1_val**2/2)
            + q * S * np.exp(-q*T) * norm.cdf(d1_val)
            - r * K * np.exp(-r*T) * norm.cdf(d2_val)
        )
    elif call_put_type == CallPutType.Put:
        theta = (
            -(S*sigma*np.exp(-q*T)) / (2*np.sqrt(2*np.pi*T)) * np.exp(-d1_val**2/2) 
            - q * S * np.exp(-q*T) * norm.cdf(-d1_val) 
            + r * K * np.exp(-r*T) * norm.cdf(-d2_val)
        )
    else:
        raise ValueError("Invalid call_put_type")
    return theta

In [24]:
## Option Parameters. ##
S0 = 54
K = 50
T = 1
r = 0.0375
q = 0.01
sigma = 0.29


V_BS = bs_put(S0, K, T, r, q, sigma)
Delta_BS = delta_bs_european(S0, K, T, r, q, sigma, CallPutType.Put) 
Gamma_BS = gamma_bs_european(S0, K, T, r, q, sigma, CallPutType.Put)
Theta_BS = theta_bs_european(S0, K, T, r, q, sigma, CallPutType.Put)

black_scholes_res = pd.DataFrame({
    "V_BS": V_BS,
    "Delta_BS": Delta_BS,
    "Gamma_BS": Gamma_BS,
    "Theta_BS": Theta_BS,
}, index=[0])
print(black_scholes_res)

       V_BS  Delta_BS  Gamma_BS  Theta_BS
0  3.579428 -0.303654    0.0222  -2.13695


## Binomial Tree Methods for European Options

In [3]:
def delta_1(V_1_0, V_1_1, S_1_0, S_1_1):
    return (V_1_0 - V_1_1) / (S_1_0 - S_1_1)

def gamma_1(V_2_0, V_2_1, V_2_2, S_2_0, S_2_1, S_2_2):
    upper_1 = (V_2_0 - V_2_1) / (S_2_0 - S_2_1)
    upper_2 = (V_2_1 - V_2_2) / (S_2_1 - S_2_2)
    lower = (S_2_0 - S_2_2) / 2
    return (upper_1 - upper_2) / lower

def theta_1(V_2_1, V_0_0, T, N):
    dt = T/N
    return (V_2_1 - V_0_0) / (2*dt)

In [4]:
def generate_geometric_sequence(start, ratio, length):
    """
    e.g. start = 10, ratio = 2, length = 5
    return [10, 20, 40, 80, 160]
    """
    progression = np.empty(length, dtype=int)
    for i in range(0, length):
        curr_term = start * pow(ratio, i)
        progression[i] = int(curr_term)
    return progression

In [ ]:
# Import numpy library
import numpy as np

# Define a function to price the American put option using the binomial tree method
def american_put_binomial(S, K, r, T, sigma, N):
    # Calculate the time step size
    dt = T / N

    # Calculate the up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u

    # Calculate the risk-neutral probabilities
    p = (np.exp(r * dt) - d) / (u - d)
    q = 1 - p

    # Initialize the stock price tree
    S_tree = np.zeros((N + 1, N + 1))
    S_tree[0, 0] = S

    # Populate the stock price tree by looping over the time steps and nodes
    for i in range(1, N + 1):
        for j in range(i + 1):
            S_tree[j, i] = S_tree[0, 0] * (u ** (i - j)) * (d ** j)

    # Initialize the option value tree
    V_tree = np.zeros((N + 1, N + 1))

    # Populate the option value tree by looping backwards from maturity to the present
    for i in range(N + 1):
        V_tree[i, N] = max(K - S_tree[i, N], 0) # intrinsic value at maturity

    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            V_tree[j, i] = max(K - S_tree[j, i], np.exp(-r * dt) * (p * V_tree[j, i + 1] + q * V_tree[j + 1, i + 1])) # maximum of exercise and continuation value

    # Return the option value at the root node
    return V_tree[0, 0]


In [25]:
def u_(T, sigma, N):
    """S1 = S0 * u_"""
    dt = T/N
    return np.exp(sigma*np.sqrt(dt))

def d_(T, sigma, N):
    """S1 = S0 * d_"""
    return 1 / u_(T, sigma, N)

def payoffs(V, i, S_temp, K, flag1, flag2):
    if not flag1 and not flag2: # European call
        V[i] = max(S_temp - K, 0)
    elif flag1 and flag2: # American put
        V[i] = max(K - S_temp, 0)
    elif flag1: # European put
        V[i] = max(K - S_temp, 0)
    else: # American call
        V[i] = max(S_temp - K, 0)

def binomial_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put
    ## Flag 2 --> 0 for European, 1 for American
    
    ## Define parameters.
    dt = T/N ## Time step.
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    d_bar = d**2
    u_bar = u**2
    q_up = (np.exp((r-q)*dt) - d) / (u - d)
    q_down = 1 - q_up
    disc = np.exp(-r*dt)
    discp = disc*q_up
    disc1p = disc*q_down
    mid = N//2
    if not N % 2:
        S_temp1 = S0
    else:
        S_temp1 = S0*d
    S_temp2 = S_temp1*u_bar
    V = np.empty(N+1)

    ## Calculate terminal payoffs.
    for i in range(mid,-1,-1):
        payoffs(V, i, S_temp1, K, flag1, flag2)
        S_temp1 *= d_bar
            
    for i in range(mid+1,N+1):
        payoffs(V, i, S_temp2, K, flag1, flag2)
        S_temp2 *= u_bar
    
    ## Work backwards through the tree.
    for j in range(N - 1, -1, -1):
        if flag2:
            S_temp = S0 * d**j
        for k in range(j + 1):
            V[k] = discp * V[k+1] + disc1p * V[k]
            
            ## American Option Adjustment.
            if flag2: 
                if not flag1: ## American call.
                    V[k] = max(V[k], S_temp - K)
                else: ## American put.
                    V[k] = max(V[k], K - S_temp)
                S_temp = S_temp * u_bar
    return V[0]

def binomial_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop):
    """
    This is a modication of Kevin's binomial pricer
    In order to get V_1_0, V_1_1, V_2_0, V_2_1, V_2_2, ...
    Args:
        early_stop: the nth step from V_0
    """
    ## Flag 1 --> 0 for call, 1 for put
    ## Flag 2 --> 0 for European, 1 for American
    
    ## Define parameters.
    dt = T/N ## Time step.
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    d_bar = d**2
    u_bar = u**2
    q_up = (np.exp((r-q)*dt) - d) / (u - d)
    q_down = 1 - q_up
    disc = np.exp(-r*dt)
    discp = disc*q_up
    disc1p = disc*q_down
    mid = N//2
    if not N % 2:
        S_temp1 = S0
    else:
        S_temp1 = S0*d
    S_temp2 = S_temp1*u_bar
    V = np.empty(N+1)

    ## Calculate terminal payoffs.
    for i in range(mid,-1,-1):
        payoffs(V, i, S_temp1, K, flag1, flag2)
        S_temp1 *= d_bar
            
    for i in range(mid+1,N+1):
        payoffs(V, i, S_temp2, K, flag1, flag2)
        S_temp2 *= u_bar
    
    ## Work backwards through the tree.
    for j in range(N - 1, -1+early_stop, -1):
        if flag2:
            S_temp = S0 * d**j
        for k in range(j + 1):
            V[k] = discp * V[k+1] + disc1p * V[k]
            
            ## American Option Adjustment.
            if flag2: 
                if not flag1: ## American call.
                    V[k] = max(V[k], S_temp - K)
                else: ## American put.
                    V[k] = max(V[k], K - S_temp)
                S_temp = S_temp * u_bar
    return V[:(early_stop+1)]

In [11]:
def approximation_error(reference, target):
    return abs(reference-target)

def linear_approximation_error(reference, target, N):
    return N * approximation_error(reference, target)

def quadratic_approximation_error(reference, target, N):
    return N**2 * approximation_error(reference, target)

### Binomial Tree

In [23]:
# Binomial Tree
# number of steps to use 
N = generate_geometric_sequence(10, 2, 8) # {10, 20, 40, ..., 1280}

# Arrays for European Put Option Pricing
EP_BT = np.empty(len(N))
EP_BT_approximation_error = np.empty(len(N))
EP_BT_linear_approximation_error = np.empty(len(N))
EP_BT_quadratic_approximation_error = np.empty(len(N))

EP_BT_delta = np.empty(len(N))
EP_BT_delta_error = np.empty(len(N))
EP_BT_gamma = np.empty(len(N))
EP_BT_gamma_error = np.empty(len(N))
EP_BT_theta = np.empty(len(N))
EP_BT_theta_error = np.empty(len(N))

for i in range(len(N)):
    # preparation
    V_1_0, V_1_1 = binomial_pricer_early_stop(
        S0, K, T, r, q, sigma, N[i], CallPutType.Put, EuroAmerType.EuropeanOption,
        early_stop=1
    )
    V_2_0, V_2_1, V_2_2 = binomial_pricer_early_stop(
        S0, K, T, r, q, sigma, N[i], CallPutType.Put, EuroAmerType.EuropeanOption,
        early_stop=2
    )
    V_0_0 = binomial_pricer(S0, K, T, r, q, sigma, N[i], CallPutType.Put, EuroAmerType.EuropeanOption)
    u = u_(T, sigma, N[i])
    d = d_(T, sigma, N[i])
    S_1_0 = S0 * u
    S_1_1 = S0 * d
    S_2_0 = S0 * u**2
    S_2_1 = S0 * u * d
    S_2_2 = S0 * d**2
    
    # value from binomial tree pricer
    EP_BT[i] = binomial_pricer(S0, K, T, r, q, sigma, N[i], CallPutType.Put, EuroAmerType.EuropeanOption)
    
    # approximation error
    EP_BT_approximation_error[i] = approximation_error(EP_BT[i], V_BS)
    EP_BT_linear_approximation_error[i] = linear_approximation_error(EP_BT[i], V_BS, N[i])
    EP_BT_quadratic_approximation_error[i] = quadratic_approximation_error(EP_BT[i], V_BS, N[i])
    
    EP_BT_delta[i] = delta_1(V_1_0, V_1_1, S_1_0, S_1_1)
    EP_BT_delta_error[i] = approximation_error(EP_BT_delta[i] , Delta_BS)
    EP_BT_gamma[i] = gamma_1(V_2_0, V_2_1, V_2_2, S_2_0, S_2_1, S_2_2)
    EP_BT_gamma_error[i] = approximation_error(EP_BT_gamma[i], Gamma_BS)
    EP_BT_theta[i] = theta_1(V_2_1, V_0_0, T, N[i])
    EP_BT_theta_error[i] = approximation_error(EP_BT_theta[i], Theta_BS)
    
binomial_tree_res = pd.DataFrame({
    "N": N,
    "V(N)": EP_BT,
    "|V(N) - V_BS|": EP_BT_approximation_error, 
    "N|V(N) - V_BS|": EP_BT_linear_approximation_error,
    "N^2|V(N) - V_BS|": EP_BT_quadratic_approximation_error,
    "Delta_1": EP_BT_delta,
    "|Delta_1 - Delta_BS|": EP_BT_delta_error,
    "Gamma_1": EP_BT_gamma, 
    "|Gamma_1 - Gamma_BS|":EP_BT_gamma_error,
    "Theta_1": EP_BT_theta,
    "|Theta_1 - Theta_BS|": EP_BT_theta_error,
})

binomial_tree_res

,N,V(N),|V(N) - V_BS|,N|V(N) - V_BS|,N^2|V(N) - V_BS|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.703631,0.124202,1.242025,12.420250,0.306502,0.610156,0.011641,0.010559,-2.227566,0.090616
1,20,3.642832,0.063404,1.268083,25.361669,0.304525,0.608179,0.011277,0.010923,-2.178015,0.041065
2,40,3.582311,0.002882,0.115300,4.611987,0.303623,0.607277,0.011238,0.010962,-2.172174,0.035224
3,80,3.582365,0.002936,0.234918,18.793401,0.303807,0.607461,0.011089,0.011111,-2.154212,0.017262
4,160,3.585679,0.006251,1.000199,160.031855,0.303748,0.607402,0.011000,0.011199,-2.142895,0.005945
5,320,3.583016,0.003588,1.148124,367.399785,0.303721,0.607375,0.010975,0.011225,-2.139740,0.002790
6,640,3.581148,0.001719,1.100460,704.294342,0.303686,0.607339,0.010965,0.011235,-2.138379,0.001429
7,1280,3.579952,0.000524,0.671047,858.940688,0.303663,0.607317,0.010960,0.011240,-2.137825,0.000876


### Average Binomial Tree

In [32]:
def avg_binomial_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.

    return (binomial_pricer(S0, K, T, r, q, sigma, N, flag1, flag2) + binomial_pricer(S0, K, T, r, q, sigma, N-1, flag1, flag2))/2 ## Simple average.


def avg_binomial_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.

    return (
        binomial_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop) 
        + binomial_pricer_early_stop(S0, K, T, r, q, sigma, N-1, flag1, flag2, early_stop)
    )/2 ## Simple average.

In [33]:
# Average Binomial Tree
# number of steps to use 
N = generate_geometric_sequence(10, 2, 8) # {10, 20, 40, ..., 1280}

# Arrays for European Put Option Pricing
EP_ABT = np.empty(len(N))
EP_ABT_approximation_error = np.empty(len(N))
EP_ABT_linear_approximation_error = np.empty(len(N))
EP_ABT_quadratic_approximation_error = np.empty(len(N))

EP_ABT_delta = np.empty(len(N))
EP_ABT_delta_error = np.empty(len(N))
EP_ABT_gamma = np.empty(len(N))
EP_ABT_gamma_error = np.empty(len(N))
EP_ABT_theta = np.empty(len(N))
EP_ABT_theta_error = np.empty(len(N))

for i in range(len(N)):
    # preparation
    V_1_0, V_1_1 = avg_binomial_pricer_early_stop(
        S0, K, T, r, q, sigma, N[i]+1, CallPutType.Put, EuroAmerType.EuropeanOption,
        early_stop=1
    )
    V_2_0, V_2_1, V_2_2 = avg_binomial_pricer_early_stop(
        S0, K, T, r, q, sigma, N[i]+1, CallPutType.Put, EuroAmerType.EuropeanOption,
        early_stop=2
    )
    V_0_0 = avg_binomial_pricer(
        S0, K, T, r, q, sigma, N[i]+1, CallPutType.Put, EuroAmerType.EuropeanOption
    )
    u = u_(T, sigma, N[i])
    d = d_(T, sigma, N[i])
    S_1_0 = S0 * u
    S_1_1 = S0 * d
    S_2_0 = S0 * u**2
    S_2_1 = S0 * u * d
    S_2_2 = S0 * d**2
    
    # value from binomial tree pricer
    EP_ABT[i] = avg_binomial_pricer(S0, K, T, r, q, sigma, N[i], CallPutType.Put, EuroAmerType.EuropeanOption)
    
    # approximation error
    EP_ABT_approximation_error[i] = approximation_error(EP_ABT[i], V_BS)
    EP_ABT_linear_approximation_error[i] = linear_approximation_error(EP_ABT[i], V_BS, N[i])
    EP_ABT_quadratic_approximation_error[i] = quadratic_approximation_error(EP_ABT[i], V_BS, N[i])
    
    EP_ABT_delta[i] = delta_1(V_1_0, V_1_1, S_1_0, S_1_1)
    EP_ABT_delta_error[i] = approximation_error(EP_ABT_delta[i] , Delta_BS)
    EP_ABT_gamma[i] = gamma_1(V_2_0, V_2_1, V_2_2, S_2_0, S_2_1, S_2_2)
    EP_ABT_gamma_error[i] = approximation_error(EP_ABT_gamma[i], Gamma_BS)
    EP_ABT_theta[i] = theta_1(V_2_1, V_0_0, T, N[i])
    EP_ABT_theta_error[i] = approximation_error(EP_ABT_theta[i], Theta_BS)
    
average_binomial_tree_res = pd.DataFrame({
    "N": N,
    "V(N)": EP_ABT,
    "|V(N) - V_BS|": EP_ABT_approximation_error, 
    "N|V(N) - V_BS|": EP_ABT_linear_approximation_error,
    "N^2|V(N) - V_BS|": EP_ABT_quadratic_approximation_error,
    "Delta_1": EP_ABT_delta,
    "|Delta_1 - Delta_BS|": EP_ABT_delta_error,
    "Gamma_1": EP_ABT_gamma, 
    "|Gamma_1 - Gamma_BS|":EP_ABT_gamma_error,
    "Theta_1": EP_ABT_theta,
    "|Theta_1 - Theta_BS|": EP_ABT_theta_error,
})

average_binomial_tree_res

,N,V(N),|V(N) - V_BS|,N|V(N) - V_BS|,N^2|V(N) - V_BS|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.623203,0.043775,0.437746,4.377459,0.297369,0.601022,0.011342,0.010858,-2.167177,0.030227
1,20,3.594339,0.014911,0.298211,5.964216,0.300565,0.604219,0.011106,0.011094,-2.146797,0.009847
2,40,3.593456,0.014028,0.561109,22.444364,0.302062,0.605716,0.011016,0.011184,-2.139979,0.003029
3,80,3.587250,0.007822,0.625780,50.062433,0.302869,0.606523,0.010984,0.011216,-2.138349,0.001399
4,160,3.583019,0.003591,0.574518,91.922843,0.303261,0.606915,0.010968,0.011232,-2.137526,0.000576
5,320,3.580991,0.001562,0.499971,159.990793,0.303449,0.607103,0.010962,0.011238,-2.137445,0.000495
6,640,3.580267,0.000839,0.536713,343.496566,0.303553,0.607206,0.010958,0.011242,-2.137160,0.000210
7,1280,3.579932,0.000504,0.644682,825.193216,0.303605,0.607258,0.010955,0.011245,-2.137001,0.000051


### Binomial Black–Scholes

In [34]:
def BBS_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.
    
    ## Define parameters.
    dt = T/N ## Time step.
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    d_bar = d**2
    u_bar = u**2
    q_up = (np.exp((r-q)*dt) - d) / (u - d)
    q_down = 1 - q_up
    disc = np.exp(-r*dt)
    discp = disc*q_up
    disc1p = disc*q_down
    mid = (N-1)//2
    if not (N-1) % 2:
        S_temp1 = S0
    else:
        S_temp1 = S0*d
    S_temp2 = S_temp1*u_bar
    V = np.empty(N)

    ## Calculate terminal payoffs.
    for i in range(mid,-1,-1):
        if not flag1: ## Call option.
            V[i] = bs_call(S_temp1, K, dt, r, q, sigma)
        else: ## Put option.
            V[i] = bs_put(S_temp1, K, dt, r, q, sigma)
        S_temp1 *= d_bar
            
    for i in range(mid+1,N):
        if not flag1:
            V[i] = bs_call(S_temp2, K, dt, r, q, sigma)
        else:
            V[i] = bs_put(S_temp2, K, dt, r, q, sigma)
        S_temp2 *= u_bar
            
    ## Work backwards through the tree.
    for j in range(N - 2, -1, -1):
        if flag2: ## American option.
            S_temp = S0 * d**j
        for k in range(j + 1):
            V[k] = discp * V[k+1] + disc1p * V[k]
            
            ## American Option Adjustment.
            if flag2: 
                if not flag1: ## American call.
                    V[k] = max(V[k], S_temp - K)
                else: ## American put.
                    V[k] = max(V[k], K - S_temp)
                S_temp = S_temp * u_bar

    return V[0]

### Binomial Black–Scholes with Richardson Extrapolation

In [35]:
def BBSN_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.

    return (2 * BBS_pricer(S0, K, T, r, q, sigma, N, flag1, flag2) - BBS_pricer(S0, K, T, r, q, sigma, N//2+1, flag1, flag2))
